# Yapay Zeka Atölyesi

_This notebook is partially adapted from Aurélien Geron's [handson-ml3](https://github.com/ageron/handson-ml3) repository. [See license](https://github.com/ageron/handson-ml3/blob/main/LICENSE) for terms and conditions._

_Adapted by Abdurrahman Can (acan [at] respectgs.us) for Tefakkuh Okulu's ML Workshop._

*1 March 2024*

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/wanderner/to-ml-workshop/blob/main/Yapay_Zeka_Atolyesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

Tefakkuh Okulu'nun Yapay Zeka Atölyesine hoş geldiniz! Böyle bir çalışmayı ilk defa yapıyoruz. Bu sebeple kendimizi geliştirebilme adına atölye süresince ve sonunda geri dönüşlerinizle bizleri desteklemenizi rica ediyoruz.

**İnteraktif not defterleri:** Şu an da okumuş olduğunuz bir interaktif not defteri. Bu not defteri üzerinde yazı ve kod bir arada bulunuyor. Eğitim amaçlı kullanımlar için birebir olmakla beraber, akademik araştırmalarda ve yapay zeka sektöründe interaktif not defterleri yaygın olarak kullanılıyor. Bundan sonrasında *Jupyter Notebook* olarak anılacaktır.

**Cloud üzerinde geliştirme:** Pratiklik olması adına sizlerden bilgisayarınıza herhangi bir kurulum yapmanızı istemedik. Aşağıdaki kodları çalıştırdığımızda bu kodlar kendi bilgisayarımız üzerinde çalışmıyor. Şu an Google Colab ürününü kullanıyoruz ve siz kod çalıştırdığınızda bunlar veri merkezlerindeki (data centers) bilgisayarlarda çalıştırılıyor ve sonuçları internet üzerinden size gösteriliyor. Bu pratiğe *cloud development* (bulutta geliştirme) veya *remote development* (uzaktan geliştirme) deniyor. Tahmin edileceği üzere bu pratik de sektörde oldukça yaygın.

## Kurulum

**Yazılım kütüphaneleri:** Yazılım dünyasında sıfırdan başlayarak geliştirme yapılmaz. Yapay zeka için de 'kod' yazdığımızdan ötürü daha önce yazılan ve yayınlanan kütüphaneleri (library) kullanıyoruz. Bu sayede hızlı geliştirme yapabilmemiz mümkün oluyor.

**Açık kaynak:** Yazılım kütüphaneleri açık kaynak yöntemiyle kolektif bir şekilde geliştiriliyor. Bu sayede performans bakımından optimize edilmiş, güvenli ve stabil sistemler geliştirmek mümkün oluyor.

Aşağıda makine öğrenmesi ve yapay zekada yaygın kullanılan bazı kütüphaneleri listeledik. Her detayın anlaşılması gerekli olmasa da bu alandan bir tat almak isterseniz websitelerini gezebilir, dokümantasyonlarına göz atabilirsiniz:

*   Numpy
*   Pandas
*   Tensorflow
*   PyTorch
*   scikit-learn

Şimdi başlayabiliriz. Öncelikle kullanabilme amacıyla kütüphanelerimizi ekliyoruz.

In [ ]:
import sys

# Python >= 3.7 versiyonuna ihtiyacımız var.
assert sys.version_info >= (3, 7)

sys.version_info

Diğer kullandığımız kütüphaneler:

In [ ]:
from packaging import version

# scikit-learn, klasik makine öğrenmesinde (classical machine learning)
# kullandığımız bir kütüphanedir.
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# matplotlib, çeşitli grafikler çizmemizi ve görselleştirmeler yapmamızı sağlar.
import matplotlib.pyplot as plt

# Görsellerimizin daha düzenli görünmesi için bazı ayarlar yapıyoruz.
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

Aşağıda kodlarda çıktılarımızın daha temiz görünmesi adına birtakım hack'ler yer alıyor. Dolayısıyla bu kısmın anlaşılması gerekmiyor.

In [ ]:
# Generate and save image
from pathlib import Path

IMAGES_PATH = Path() / "images" / "classification"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Wrap long lines:
from IPython.display import HTML, display

def set_css():
  display(HTML('<style>pre {white-space: pre-wrap;}</style>'))
get_ipython().events.register('pre_run_cell', set_css)

# Yapay Zeka ile Sınıflandırma (Classification)

Yapay zekada çeşitli görev türleri bulunur:

*   Classification (Sınıflandırma)
*   Clustering (Kümelendirme)
*   Anomaly detection (Anormallik bulma)
*   Natural Language Processing (Doğal dil işleme)

Bunlardan en basiti sınıflandırma olduğu için şimdi bir sınıflandırma eğitimi gerçekleştireceğiz.

Makine öğrenmesi için aşağıdakilere ihtiyacımız var:

1. Dataset (Veriseti)
2. Model
3. Computing power (İşleme gücü)

## Veriseti: MNIST Dataset (MNIST Veriseti)

Bugün yapay zekaya başlangıçta eğitim amaçlı yaygın olarak kullanılan MNIST verisetini kullanacağız. Verisetimiz el yazılarıyla oluşturulmuş rakamlardan oluşuyor.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

MNIST verisetinin yazarları tarafından yazılan açıklama metnini görelim:

In [ ]:
print(mnist.DESCR)

In [ ]:
mnist.keys()

MNIST verisetinde çeşitli detaylar yer alıyor. Biz sadece data ve target'i kullanacağız.

Data el yazılarının verisi, target ise o el yazısının ifade ettiği sayı.

Data'yı X, target'i y olarak isimlendiriyoruz.

In [ ]:
X, y = mnist.data, mnist.target

X'i görelim:


In [ ]:
X

X'in şekline bakalım:

In [ ]:
X.shape

70 bin adet el yazısı sayı var, her el yazısı 784 farklı numarayla ifade edilmiş. Bu da 28 * 28 bir kare şekil ifade ediyor.

In [ ]:
28 * 28

In [ ]:
import matplotlib.pyplot as plt

def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = X[0]
plot_digit(some_digit)
save_fig("some_digit_plot")  # extra code
plt.show()

print(y[0])

Şimdi de y'yi görelim:

In [ ]:
y

y'nin şekli:

In [ ]:
y.shape

ve y:

In [ ]:
y[0]

Verisetini daha iyi gözlemleyebilmek adına çeşitli sayıları görelim:

In [ ]:
# extra code – this cell generates and saves Figure 3–2
plt.figure(figsize=(9, 9))
for idx, image_data in enumerate(X[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)
plt.subplots_adjust(wspace=0, hspace=0)
save_fig("more_digits_plot", tight_layout=False)
plt.show()

### Verisetinin Bölümlenmesi

Elimizde 70 bin veri bulunuyor. Bunu iki gruba bölmemiz gerekiyor:

1. **Training set (Eğitim seti):** Modelimizi eğitmek için bu seti kullanıyoruz. Eğitim süresinde 784 numaralık veriyi ve karşılığı olan sayıyı modelimize veriyoruz.
2. **Test set:** Belirli bir eğitimden süresinden sonra modelimizi test etmek için bu seti kullanıyoruz. Eğitilen modelimize elimizdeki sayıların ne olduğunu söylemeden tahmin etmesini istiyoruz. Modelin verdiği sonuçları asıl sonuçlarla karşılaştırıp ne kadar isabetli çalıştığını bu sayede belirleyebileceğiz.

Modelin tamamını eğiteceğimizi düşünürsek test için 10 bin veri ayırmamız yeterli. Eğitim içinse daha fazla veriye ihtiyacımız var. Dolayısıyla 60 bin ve 10 bin olarak ayırabiliriz. Ancak çalıştırma süresinin uzamaması adına bugün verisetinin sadece bir kısmını kullanacağız.

In [ ]:
# Tüm verisetini kullanmak için aşağıdaki iki satırı kullanın:
# X_train, X_test = X[:60000], X[60000:]
# y_train, y_test = y[:60000], y[60000:]

X_train, X_test = X[:2000], X[2000:3000]
y_train, y_test = y[:2000], y[2000:3000]

## Çok Sınıflı Sınıflandırma (Multiclass Classification)

Elimizde ikiden fazla sınıf varsa buna Multiclass Classification ismi veriyoruz. Rakam sınıflandırma yaparken 10 farklı rakamımız olduğu için elimizde 10 sınıf var.

Multiclass Classification için pek çok yöntem kullanabiliriz. Bunların en genelgeçer ve kabul görenlerinden birisi Support Vector Machines, SVM'dir. Kıcasa SVM'ler, verileri daha kolay sınıflandırabilmek için üst seviye boyutlara çevirip sınıflandırma işlemini uygular ve tekrar önceki boyuta döner. Daha teknik bilgi bugün konumuz dışı olduğundan meraklısına internet üzerinde araştırmasını tavsiye ediyoruz.

SVM'lerin çalışması veriseti büyüdükçe daha da uzar. Atölye sırasında uzun süre beklememek adına MNIST verisetimizin sadece ilk 2000 verisini eğitim için kullanacağız.

In [ ]:
from sklearn.svm import SVC

# Modelimizi her çalıştırdığımızda herkeste aynı sonucu vermesi için rastgele
# sayı üretme yöntemini sabitliyoruz.
svm_clf = SVC(random_state=42)

# fit() metodu bizim için fazlasıyla soyutlanmış olduğu için kodumuz
# oldukça temiz. Burası eğitim olarak adlandırdığımız aksiyonun gerçekleştiği
# yer:
svm_clf.fit(X_train[:2000], y_train[:2000])  # y_train, not y_train_5

Python'da her metodun dokümanını okumanın oldukça basit bir yolu var:

In [ ]:
help(svm_clf.fit)

Modelimizin sınıflarını görelim:

In [ ]:
svm_clf.classes_

Verisetimizden sadece bir örneği tahmin ettirelim:

In [ ]:
some_digit = X[0]

svm_clf.predict([some_digit])

10 tane sınıfımız olduğu için modelimiz aslında 10 tane skor veriyor:

In [ ]:
some_digit_scores = svm_clf.decision_function([some_digit])
some_digit_scores.round(2)

Ve biz bu skorlardan en yüksek olanı alıyoruz:

In [ ]:
class_id = some_digit_scores.argmax()
class_id

Sınıflarımızdan o indeksteki değeri görelim. Bu bizim tahmin sonucumuz:

In [ ]:
svm_clf.classes_[class_id]

Son olarak tüm test verisetimizi verip modelin skorunu görelim:

In [ ]:
svm_clf.score(X_test, y_test)

# Transfer Learning (Öğrenim Transferi)

Natural Language Processing (NLP - Doğal Dil İşleme) 2017'de yayınlanan Transformer makalesiyle tamamen değişime uğradı. Bu alanda artık çok büyük miktarda veriyi (metin) daha hızlı bir şekilde işlemek mümkün oldu.

Klasik NLP'de her farklı görev için yeni bir model eğitilirken günümüzde çok büyük miktarlarda veriler toplanıp devasa modeller eğitiliyor. Daha sonra bu modeller çeşitli amaçlar için fine-tune (son halini verme) işleminden geçirilip kullanıma sunuluyor.

Bugün çeşitli yapay zeka görevleri için örnekler deneyeceğiz. Bunlar NLP alanından olacak, çünkü görüntü işlemek daha çok zaman ve işlem gücü gerektiriyor.

**HuggingFace🤗:** [Hugging Face](https://huggingface.co), transformer temelli yapay zeka modellerini ve verisetlerini bir araya getiren bir şirket ve açık kaynak topluluğu. Araştırmacılar ve yapay zeka şirketleri akademik veya iş amaçlı geliştirdikleri model ve verisetlerini Hugging Face üzerinde yayınlıyorlar. Kolaylık adına biz de bugün [Hugging Face modelleri](https://huggingface.co/facebook/bart-large-cnn) üzerinde denemeler yapacağız.

## Text Summarization (Metin Özetleme)

Facebook tarafından yayınlanan BART (*BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension, [makale](https://arxiv.org/abs/1910.13461)*) modeliyle metin özetleme denemeleri yapalım.

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Ön eğitimden geçirilmiş modeli indirelim.
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def generate_summary(text):
    '''BART modelini kullanarak özet üreten model fonksiyonu.'''

    # Metinleri matematiksel hale çevirelim.
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)

    # Modeli kullanarak özeti üretelim.
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=30, max_length=200, early_stopping=True)

    # Model bize matematiksel bir çıktı veriyor.
    # Bu çıktıyı tekrar metin haline çevirelim.
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return summary

Yukarıdaki hücreyi her çalıştırdığımızda modeli internetten indireceği için bunu önleyip denemelerimizi aşağıdaki hücrede gerçekleştirelim.

In [ ]:
# Example text to summarize
input_text = """
Your input text goes here. This can be a long document or an article that you want to summarize.
"""

# Generate the summary
summary = generate_summary(input_text)
print("Generated Summary:")
print(summary)

## Translation (Metin Çevirisi)

Google'ın yayınlamış olduğu T5 modeli (*T5: Text-To-Text Transfer Transformer, [makale](https://arxiv.org/abs/1910.10683), [kod](https://github.com/google-research/text-to-text-transfer-transformer)*) ile metin çevirisi deneyelim.

Zaman tasarrufu adına `t5-small` veya `t5-base` modelini kullanıyoruz. Daha başarılı sonuçlar için `t5-large`, `t5-3b` veya `t5-5b` de kullanılabilir. Aşağıdaki kodda `t5-small` ifadelerini değiştirerek deneyebilirsiniz.

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Ön eğitimden geçirilmiş modeli indirelim.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def translate_text(input_text, target_language="fr"):
    '''T5 modelini kullanarak çeviri yapan model fonksiyonu.'''

    # Metinleri matematiksel hale çevirelim.
    input_text = "translate English to " + target_language + ": " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Modeli çalıştıralım.
    translation_ids = model.generate(inputs.input_ids, max_length=512, num_beams=4, early_stopping=True)

    # Model bize matematiksel bir çıktı veriyor.
    # Bu çıktıyı tekrar metin haline çevirelim.
    translation = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translation

Çevirileri test edelim:

In [ ]:
# Example text for translation
input_text = "Hello, how are you?"

# Translate the text to French
translated_text = translate_text(input_text, target_language="fr")
print("Translated Text:")
print(translated_text)

## Text Generation (Metin Üretimi)

Son olarak transformer'lar ile metin üretimini deneyelim.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# Ön eğitimden geçirilmiş modeli indirelim.
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

def generate_text(prompt, max_length=100):
    # Metinleri matematiksel hale çevirelim.
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Modeli çalıştıralım.
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                            no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7,
                            do_sample=True)

    # Çıktıyı tekrar metin haline çevirelim.
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

Modeli test edelim:

In [ ]:

# Example prompt for text generation
prompt = "Once upon a time, in a land far away, there lived a"

# Generate text based on the prompt
generated_text = generate_text(prompt, max_length=150)
print("Generated Text:")
print(generated_text)

# Son Notlar

Bu atölyede yapay zekanın yapabileceklerinin bir kısmını gördük ve temellerini biraz inceledik. Son 15 yılda ciddi gelişmeeler yaşanmış olan yapay zekanın kapasiteleri bundan çok daha derin ve bu alan akademide oldukça geniş.